In [2]:
def highlight_best(data):
    best_color = 'green'
    second_best_color = 'orange'
    best_attr = f'background-color: {best_color}'
    second_best_attr = f'background-color: {second_best_color}; color: black'
    
    def extract_value(val):
        # if string
        if isinstance(val, str):
            return float(val.split('±')[0].strip())
        else:
            return val
    
    if data.ndim == 1:  # Single row or column (Series)
        # Highlight based on whether the metric prefers min or max
        col_name = data.name.lower()
        if 'loss' in col_name or 'fid' in col_name:
            sorted_data = data.apply(extract_value).sort_values()
        else:
            sorted_data = data.apply(extract_value).sort_values(ascending=False)
        best = sorted_data.iloc[0]
        second_best = sorted_data.iloc[1]
        return [best_attr if v == best else second_best_attr if v == second_best else '' for v in data.apply(extract_value)]
    else:  # DataFrame case
        styled_df = pd.DataFrame('', index=data.index, columns=data.columns)
        for col in data.columns:
            col_name = col.lower()
            if 'loss' in col_name or 'fid' in col_name:
                sorted_data = data[col].apply(extract_value).sort_values()
            else:
                sorted_data = data[col].apply(extract_value).sort_values(ascending=False)
            best = sorted_data.iloc[0]
            second_best = sorted_data.iloc[1]
            styled_df.loc[data[col].apply(extract_value) == best, col] = best_attr
            styled_df.loc[data[col].apply(extract_value) == second_best, col] = second_best_attr
        return styled_df

In [8]:
import pandas as pd

csv_file = "/home/siangling/B-LoRA/evaluation_results.xlsx"

df = pd.read_excel(csv_file)
metrics = df.columns[1:]
df.style.apply(highlight_best, subset=metrics, axis=0)

,method,style_loss,style_similarity,content_loss,content_similarity,fid
0,original,0.995 ± 1.692,0.355 ± 0.061,19.372 ± 7.782,0.150 ± 0.031,26.659 ± 31.600
1,lab_transfer,1.038 ± 1.691,0.350 ± 0.054,18.030 ± 6.160,0.151 ± 0.028,39.869 ± 64.897
2,luv_transfer,0.943 ± 1.608,0.352 ± 0.052,18.105 ± 6.529,0.153 ± 0.031,32.842 ± 44.045
3,pca_transfer,0.944 ± 1.817,0.358 ± 0.054,17.933 ± 6.389,0.156 ± 0.033,38.342 ± 68.599
